In [12]:
import numpy as np

from numpy import save

import cv2 as cv

import glob

import os

import shutil

from numpy import load


shutil.rmtree('result_images')

os.makedirs('result_images')



result_images = "result_images"

filepath = "SampleImagesWorking"

orig_files = [file for file in glob.glob(filepath+"/*.jpg")]

new_files = [os.path.join(result_images, os.path.basename(f)) for f in orig_files]



obj_points = []

img_points = []

cnt = 0

num = 0



criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

obj_p = np.zeros((6*6, 3), np.float32)

obj_p[:, :2] = np.mgrid[0:6, 0:6].T.reshape(-1, 2)

for orig_f, new_f in zip(orig_files, new_files):

    print('\r', "Current Image = " + str(num), end='')

    num += 1

    img = cv.imread(orig_f)

    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    ret, corners = cv.findChessboardCorners(gray, (6,6), None)

    print(gray.shape[::-1])

    if ret:

        cnt += 1
        print("Count = "+ str(cnt))

        obj_points.append(obj_p)

        corners2 = cv.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)

        img_points.append(corners)
        
        cv.drawChessboardCorners(img, (6,6), corners2, ret)

        cv.imwrite(new_f, img)



print("\n Number of Correct Images = " + str(cnt))

ret, mtx, dist, r_vectors, t_vectors = cv.calibrateCamera(obj_points, img_points, (1680, 1050), None, None)
    
print("\nThe Camera Matrix is = \n " + str(mtx))
    
print("\nThe distortion Coefficient are = \n " + str(dist))
    
save('mtx.npy', mtx)
    
save('dist.npy', dist)


#Obtain Undistorted Images
mtx = load('mtx.npy')

dist = load('dist.npy')


shutil.rmtree('undistorted_images')

os.makedirs('undistorted_images')

result_images = "undistorted_images"

filepath = "SampleImagesWorking"

orig_files = [file for file in glob.glob(filepath+"/*.jpg")]

new_files = [os.path.join(result_images, os.path.basename(f)) for f in orig_files]

for orig_f, new_f in zip(orig_files, new_files):

    img = cv.imread(orig_f)

    h, w = img.shape[:2]

    new_camera_mtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w, h), 0, (w, h))

    dst = cv.undistort(img, mtx, dist, None, new_camera_mtx)

    cv.imwrite(new_f, dst)

cv.destroyAllWindows()

# Reprojection Error
mean_error = 0

for i in range(len(obj_points)):

    imgpoints2, _ = cv.projectPoints(obj_points[i], r_vectors[i], t_vectors[i], mtx, dist)

    error = cv.norm(img_points[i], imgpoints2, cv.NORM_L2)/len(imgpoints2)

    mean_error += error

print( "\nTotal Error : {}".format(mean_error/len(obj_points)) )  










 Current Image = 0(1920, 1080)
Count = 1
 Current Image = 1(1920, 1080)
Count = 2
 Current Image = 2(1920, 1080)
 Current Image = 3(1920, 1080)
Count = 3
 Current Image = 4(1920, 1080)
Count = 4

 Number of Correct Images = 4

The Camera Matrix is = 
 [[1.25654247e+03 0.00000000e+00 8.90733249e+02]
 [0.00000000e+00 1.25428777e+03 5.31314207e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]

The distortion Coefficient are = 
 [[-0.34347769  0.15656908 -0.00114245  0.00140549 -0.04051093]]

Total Error : 0.01566187156783607
